In [1]:
# Dependencies
import json
import matplotlib.pyplot as plt
import requests
from scipy import stats
import pandas as pd
from config import api_key

In [2]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="

In [3]:
# location of filtered loc file
loc_file = "templates/sightingsLocation.json"
# read file in
loc_df = pd.read_json(loc_file, lines=False)
# transpose df
loc_df = loc_df.transpose()

loc_df

,number,county,state,latitude,longitude,geohash,date
0,60,Skagit County,Washington,48.6406,-121.805,c29ksq8pfc,1994-05-13
1,70,Washington County,New York,None,None,None,None
2,76,Baxter County,Arkansas,36.3714,-92.2514,9yqquv0wdy,2001-08-11
3,77,Kittitas County,Washington,46.9833,-121.092,c23gk3p3ep,1983-09-01
4,80,Lane County,Oregon,43.69,-122.376,9rbbx38bup,1983-08-12
...,...,...,...,...,...,...,...
4929,51949,None,None,44.9005,-63.8457,None,2015-10-11
4930,55604,None,None,52.7807,-116.365,None,2016-09-06
4931,58445,None,None,42.3669,-73.1823,None,2017-10-30
4932,59610,None,None,50.8117,-114.783,None,2018-06-12


In [4]:
# get rid of geohash
loc_df = loc_df[['number','county','state','latitude','longitude','date']]

In [5]:
# drop rows that don't include a date
loc_df = loc_df.dropna(subset=['date'])
loc_df

,number,county,state,latitude,longitude,date
0,60,Skagit County,Washington,48.6406,-121.805,1994-05-13
2,76,Baxter County,Arkansas,36.3714,-92.2514,2001-08-11
3,77,Kittitas County,Washington,46.9833,-121.092,1983-09-01
4,80,Lane County,Oregon,43.69,-122.376,1983-08-12
6,83,Madera County,California,37.3594,-119.644,1988-10-01
...,...,...,...,...,...,...
4929,51949,None,None,44.9005,-63.8457,2015-10-11
4930,55604,None,None,52.7807,-116.365,2016-09-06
4931,58445,None,None,42.3669,-73.1823,2017-10-30
4932,59610,None,None,50.8117,-114.783,2018-06-12


In [6]:
# drop rows that don't have at least two of county state latitude longitude
# to use to find location with the weathermap api
loc_df.dropna(thresh=3)

,number,county,state,latitude,longitude,date
0,60,Skagit County,Washington,48.6406,-121.805,1994-05-13
2,76,Baxter County,Arkansas,36.3714,-92.2514,2001-08-11
3,77,Kittitas County,Washington,46.9833,-121.092,1983-09-01
4,80,Lane County,Oregon,43.69,-122.376,1983-08-12
6,83,Madera County,California,37.3594,-119.644,1988-10-01
...,...,...,...,...,...,...
4929,51949,None,None,44.9005,-63.8457,2015-10-11
4930,55604,None,None,52.7807,-116.365,2016-09-06
4931,58445,None,None,42.3669,-73.1823,2017-10-30
4932,59610,None,None,50.8117,-114.783,2018-06-12


In [7]:
# drop rows that don't include a date
loc_df = loc_df.dropna(subset=['latitude','longitude'])
loc_df

,number,county,state,latitude,longitude,date
0,60,Skagit County,Washington,48.6406,-121.805,1994-05-13
2,76,Baxter County,Arkansas,36.3714,-92.2514,2001-08-11
3,77,Kittitas County,Washington,46.9833,-121.092,1983-09-01
4,80,Lane County,Oregon,43.69,-122.376,1983-08-12
6,83,Madera County,California,37.3594,-119.644,1988-10-01
...,...,...,...,...,...,...
4929,51949,None,None,44.9005,-63.8457,2015-10-11
4930,55604,None,None,52.7807,-116.365,2016-09-06
4931,58445,None,None,42.3669,-73.1823,2017-10-30
4932,59610,None,None,50.8117,-114.783,2018-06-12


In [10]:
from datetime import datetime
import time

In [16]:
loc_df[['year','month','day']]=loc_df.date.str.split("-",expand=True)
loc_df

,number,county,state,latitude,longitude,date,year,month,day
0,60,Skagit County,Washington,48.6406,-121.805,1994-05-13,1994,05,13
2,76,Baxter County,Arkansas,36.3714,-92.2514,2001-08-11,2001,08,11
3,77,Kittitas County,Washington,46.9833,-121.092,1983-09-01,1983,09,01
4,80,Lane County,Oregon,43.69,-122.376,1983-08-12,1983,08,12
6,83,Madera County,California,37.3594,-119.644,1988-10-01,1988,10,01
...,...,...,...,...,...,...,...,...,...
4929,51949,None,None,44.9005,-63.8457,2015-10-11,2015,10,11
4930,55604,None,None,52.7807,-116.365,2016-09-06,2016,09,06
4931,58445,None,None,42.3669,-73.1823,2017-10-30,2017,10,30
4932,59610,None,None,50.8117,-114.783,2018-06-12,2018,06,12


In [21]:
for row in loc_df:
    dt = datetime(loc_df['year'],loc_df['month'],loc_df['day'])
    timestamp = dt.replace(tzinfor=timezone.utc).timestamp()
    loc_df['unix'].append(timestamp)

TypeError: cannot convert the series to <class 'int'>